# Papermill for running batch data analysis using the template pipelines


## Overview

* Setup: load packages/setup path
* Load calibration h5 file (genreated by calibration pipeline) to load Mask and beamline parameters
* Check one data and convert scattering data to q-phi map, q-Iq, ang-Iang
* Find the interested dataset and do batch reduction
* Produce images (png format)
* Export results to a hdf file 
 

### DEV
* V0: Initialize the pipeline (2019/10/26 by YG yuzhang@bnl.gov)


 

## TODLIST
* 1): Put codes into a package

# Load package

In [1]:
%matplotlib notebook
from pyScatt.packages import *
plt.rcParams.update({'figure.max_open_warning': 0})
plt.rcParams.update({ 'image.origin': 'lower'   })
plt.rcParams.update({ 'image.interpolation': 'none'   })

In [2]:
%run -i /home/yuzhang/Software/pyScatt/pyScatt/generic_functions.py

# Setup paths

In [4]:
####
username = 'yuzhang'


inDir = None
outDir = None #'/home/group/NSLSII_Data/SMI/2020_Cycle2/304231_Dinca/Analysis_Results/Yugang/SAXS/ResPipes/'



T=True
F=False

In [5]:
t = get_current_time( pattern = '%Y_%m_%d_%H' )
print( t )

2020_11_08_16


# Papermill functions

In [8]:
path =  '/home/yuzhang/XScattering/2020_2/SMI/Pipelines/Yugang/'
path = '/home/yuzhang/XScattering/2020_3/CMS/Pipelines/Dishari/'
path = '/home/yuzhang/XScattering/2020_3/SMI/Pipelines/CWang/'
path = '/home/yuzhang/XScattering/2020_3/SMI/Pipelines/Brian/'
path =  '/home/yuzhang/XScattering/2020_3/SMI/Pipelines/Yugang/'
path =  '/home/yuzhang/XScattering/2020_3/SMI/Pipelines/Dinca/'


outDir = path

def _run_data_reduction_waxs( uid_list=None,          
        template_pipeline = path + 'General_Data_Reduction_Template_SMI_WAXS_V1.ipynb',                
         ):
    
    ''' YG. Octo 26, 2019, Data reduction for SAXS/WAXS data using SAXS_Data_Reduction_Template_V0 pipeline
    Input:
        uid_list: list, a list of strings (the uique data id or filename)
        template_pipeline: str, the filename of the template pipeline
        output_pipeline:str, the filename of the output pipeline
    Output:
        None
    '''     
    t = get_current_time( pattern = '%Y_%m_%d_%H' )
    output_pipeline = outDir + 'res_pipeline_%s_waxs.ipynb'%t
    pm.execute_notebook(
                        template_pipeline, output_pipeline,         
                        parameters = dict( uid_list = uid_list ),
                        kernel_name='python3', report_mode=False 
                         )      
    
def _run_data_reduction_saxs( uid_list=None,          
        template_pipeline = path + 'General_Data_Reduction_Template_SMI_SAXS_V1.ipynb',                
         ):
    
    ''' YG. Octo 26, 2019, Data reduction for SAXS/WAXS data using SAXS_Data_Reduction_Template_V0 pipeline
    Input:
        uid_list: list, a list of strings (the uique data id or filename)
        template_pipeline: str, the filename of the template pipeline
        output_pipeline:str, the filename of the output pipeline
    Output:
        None
    '''     
    t = get_current_time( pattern = '%Y_%m_%d_%H' )
    output_pipeline = outDir + 'res_pipeline_%s_saxs.ipynb'%t
    pm.execute_notebook(
                        template_pipeline, output_pipeline,         
                        parameters = dict( uid_list = uid_list ),
                        kernel_name='python3', report_mode=False 
                         ) 
    
    
def _run_get_Sq( uid_list,          
        template_pipeline = path + 'SAXS_Data_Analysis_SQ_Template_V2.ipynb',                
        output_pipeline = outDir + 'test_GetSq.ipynb' ):
    
    ''' YG. Octo 26, 2019, Data analysis for SAXS/WAXS data using SAXS_Data_Analysis_SQ_Template_V0 pipeline
    Input:
        uid_list: list, a list of strings (the uique data id or filename)
        template_pipeline: str, the filename of the template pipeline
        output_pipeline:str, the filename of the output pipeline
    Output:
        None
    '''     
    pm.execute_notebook(
                        template_pipeline, output_pipeline,         
                        parameters = dict( uid_list = uid_list ),
                        kernel_name='python3', report_mode=False 
                         )          
    
    
    

## run the data reduction

In [7]:
t0 = time.time() 
_run_data_reduction_waxs(   )    
run_time( t0 )


Total time: 1.194 min


In [16]:
if F:
    t0 = time.time() 
    _run_data_reduction_waxs(   )    
    run_time( t0 )


Total time: 39.428 min


## List all data

In [6]:
uid_list_all = sorted( ls_dir( inDir, have_list=[    '.tiff'   ], exclude_list=[ 'waxs'  ]) )
print(len(uid_list_all))
uid_list_all

10


['AgBH_cali_13.5kev_x-17.000_y-2.170_1.00s_2608570_saxs.tiff',
 'AgBH_cali_13.5kev_x-17.000_y-2.170_5.00s_2608571_saxs.tiff',
 'AgBH_cali_13.5kev_x-17.000_y-2.170_5.00s_2608571_saxs.tiff_q_Iq.csv',
 'FL_1024Morn_F1_x0.100_y0.150_0.10s_2605949_saxs.tiff',
 'FL_1024Morn_F1_x0.100_y0.150_0.10s_2605950_saxs.tiff',
 'FL_1024Ngt_10nmSP_Form_Factor_x-0.900_y-0.400_5.00s_2606361_saxs.tiff',
 'FL_1026Morn_F2_x0.000_y0.000_1.00s_2607309_saxs.tiff',
 'f3b03825-edfa-43f3-b333_000001.tiff',
 'f3b03825-edfa-43f3-b333_000002.tiff',
 'f3b03825-edfa-43f3-b333_000003.tiff']

## Run the template pipelines for batch data analysis

* ## For data reduction

* #### Find the interested data

In [11]:
uid_list1 = sorted( ls_dir( inDir, have_list=[    'F1',  ], exclude_list=['waxs', 'AgBH']) )
uid_list2 = sorted( ls_dir( inDir, have_list=[    'F2',  ], exclude_list=['waxs', 'AgBH']) )
uid_list3 = sorted( ls_dir( inDir, have_list=[    'f3b' ,], exclude_list=['waxs', 'AgBH']) )
uid_list4 = sorted( ls_dir( inDir, have_list=[    '10nmSP' ,], exclude_list=['waxs', 'AgBH']) )
uid_list =  list( np.concatenate( [uid_list1,uid_list2,uid_list3, uid_list4   ] ) )

#uid_list = uid_list_all

print(len(uid_list))

7


In [12]:
uid_list

['FL_1024Morn_F1_x0.100_y0.150_0.10s_2605949_saxs.tiff',
 'FL_1024Morn_F1_x0.100_y0.150_0.10s_2605950_saxs.tiff',
 'FL_1026Morn_F2_x0.000_y0.000_1.00s_2607309_saxs.tiff',
 'f3b03825-edfa-43f3-b333_000001.tiff',
 'f3b03825-edfa-43f3-b333_000002.tiff',
 'f3b03825-edfa-43f3-b333_000003.tiff',
 'FL_1024Ngt_10nmSP_Form_Factor_x-0.900_y-0.400_5.00s_2606361_saxs.tiff']

In [15]:
t0 = time.time() 
_run_data_reduction( uid_list )    
run_time( t0 )


Total time: 37.435 sec


* ## For Get Sq

* #### Find the interested data

In [19]:
uid_list = sorted( ls_dir( outDir, have_list=[    'h5',  ], exclude_list=['waxs', 'AgBH', 'png', 'test', 'Free',
                                                                          't65', 'csv',
                                                                          'Form_Factor', 'setup']) )
print(len(uid_list))

6


In [20]:
uid_list

['FL_1024Morn_F1_x0.100_y0.150_0.10s_2605949_saxs.tiff.h5',
 'FL_1024Morn_F1_x0.100_y0.150_0.10s_2605950_saxs.tiff.h5',
 'FL_1026Morn_F2_x0.000_y0.000_1.00s_2607309_saxs.tiff.h5',
 'f3b03825-edfa-43f3-b333_000001.tiff.h5',
 'f3b03825-edfa-43f3-b333_000002.tiff.h5',
 'f3b03825-edfa-43f3-b333_000003.tiff.h5']

In [21]:
t0 = time.time() 
_run_get_Sq( uid_list )   
run_time( t0 )


Total time: 26.399 sec


# The End